In [2]:
import os

In [3]:
%pwd

"c:\\Users\\Fahad's WorkStation\\Programs\\PF\\DL-project-with-MLOPs\\research"

In [4]:
os.chdir("../")

In [5]:
%pwd

"c:\\Users\\Fahad's WorkStation\\Programs\\PF\\DL-project-with-MLOPs"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config




In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath.replace('.keras'),  # Save as .keras
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-12-30 08:26:15,773: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-30 08:26:15,777: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-30 08:26:15,779: INFO: common: created directory at: artifacts]
[2024-12-30 08:26:15,780: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-12-30 08:26:15,781: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'artifacts\\prepare_callbacks\\checkpoint_dir\\model.h5' -> '.keras'

In [13]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"


'tb_logs_at_2024-12-30-07-54-33'